Let's see our train.csv to see what we are dealing with!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('./datasets/train.csv', index_col='id')
train_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 18.7+ MB


# **Data Cleaning**

We find that there a lot of object columns with a number of missing (NaN) values. This next section will explore data to clean and make values possible to provide meaningful parameters in our model. This includes: 
- Dropping unneccessary columns
- Converting columns that have non-numerical datatypes to numerical datatypes for analysis.
- Dropping/imputing NaN values  

In [4]:
# Number of missing values (NaN)
train_df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [5]:
unique_counts = train_df.nunique()
unique_counts

brand             57
model           1897
model_year        34
milage          6651
fuel_type          7
engine          1117
transmission      52
ext_col          319
int_col          156
accident           2
clean_title        1
price           1569
dtype: int64

In [6]:
# The misspelling really annoyed me :D
train_df.rename(columns={"milage":"mileage"}, inplace=True)
train_df

,brand,model,model_year,mileage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500
...,...,...,...,...,...,...,...,...,...,...,...,...
188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900


## Transmission

Let's explore the different types of transmission types in this data types and try to catergorize the many types into a few broad catergories.

In [7]:
train_df['transmission'].unique()

array(['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T',
       '8-Speed A/T', '10-Speed Automatic', '1-Speed A/T', '6-Speed A/T',
       '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic',
       '9-Speed Automatic', '5-Speed A/T', 'Automatic',
       '7-Speed Automatic with Auto-Shift', 'CVT Transmission',
       '5-Speed M/T', 'M/T', '6-Speed M/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T', '2-Speed A/T',
       '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T',
       '6-Speed Manual', 'Transmission Overdrive Switch',
       '8-Speed Automatic with Auto-Shift', '7-Speed Manual',
       '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
       '6-Speed Automatic with Auto-Shift',
       '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
       '8-Speed Manual', 'Manual', '–', '2', '6 Speed At/Mt',
       '5-Speed Automatic', '2-Speed Automatic', '8-SPEED A/T', '7-Speed',
       'Variable', 'Single-Speed Fixed Gear', '8-SPEED AT',


We can classify the broad catergories as follows:
- **Automatic**
- **Manual**
- **CVT**
- **Dual** (For electric vehicles)
- **Other** (This will be to cover all those transmission descriptions so that we don't ignore/drop such rows)

In [8]:
# Defining function to catergorize transmission
def catergorize_transmission(transmission):
    if 'A/T' in transmission or 'Automatic' in transmission:
        return 'Automatic'
    elif 'M/T' in transmission or 'Manual' in transmission:
        return 'Manual'
    elif 'CVT' in transmission:
        return 'CVT'
    elif 'Dual' in transmission:
        return 'Dual'
    else:
        return 'Other'

In [9]:
train_df['transmission_cat'] = train_df['transmission'].apply(catergorize_transmission)
train_df.head()

,brand,model,model_year,mileage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmission_cat
id,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,Automatic
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,Automatic
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,Automatic
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,Dual
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,Automatic


In [10]:
train_df = train_df.drop(columns='transmission')
train_df.head(5)

,brand,model,model_year,mileage,fuel_type,engine,ext_col,int_col,accident,clean_title,price,transmission_cat
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,Yellow,Gray,None reported,Yes,4200,Automatic
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,Silver,Beige,At least 1 accident or damage reported,Yes,4999,Automatic
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,Blue,Gray,None reported,Yes,13900,Automatic
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Black,Black,None reported,Yes,45000,Dual
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Black,Beige,None reported,Yes,97500,Automatic


## Engine

The engine descrptions are the most useful in this dataset that can help us obtain vital information for building our model, i.e, **Horsepower**, **Engine Size**, **Fuel Type**. 

Our existing 'fuel_type' column has many missing/NaN data points. Since our engine description covers what fuel is used, we can save operations on imputing values to 'fuel_type' and perform regex to extract our data and store them in appopriate columns. 

In [11]:
import re

def extract_engine_features(engine_str):
    
    hp_match = re.search(r'(\d+\.\d+)HP', engine_str)
    horsepower = float(hp_match.group(1)) if hp_match else None
    
    
    engine_size_match = re.search(r'(\d+\.\d+)L', engine_str)
    engine_size = float(engine_size_match.group(1)) if engine_size_match else None
    
   
    fuel_type_match = re.search(r'(Gasoline|Flex Fuel|Electric)', engine_str)
    fuel_type = fuel_type_match.group(0) if fuel_type_match else None
    
    return horsepower, engine_size, fuel_type

In [12]:
train_df[['horsepower', 'engine_size', 'type_of_fuel']] = train_df['engine'].apply(lambda x: pd.Series(extract_engine_features(x)))
train_df = train_df.drop(columns=['fuel_type','engine'])
train_df.head(4)

,brand,model,model_year,mileage,ext_col,int_col,accident,clean_title,price,transmission_cat,horsepower,engine_size,type_of_fuel
id,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Yellow,Gray,None reported,Yes,4200,Automatic,172.0,1.6,Gasoline
1,Lincoln,LS V8,2002,143250,Silver,Beige,At least 1 accident or damage reported,Yes,4999,Automatic,252.0,3.9,Gasoline
2,Chevrolet,Silverado 2500 LT,2002,136731,Blue,Gray,None reported,Yes,13900,Automatic,320.0,5.3,Flex Fuel
3,Genesis,G90 5.0 Ultimate,2017,19500,Black,Black,None reported,Yes,45000,Dual,420.0,5.0,Gasoline


## Clean Title

Clean title gives vehicles to have a higher resale value as it was certified to have little to no damges or have not been declared a total loss by an insurance company. Other vehicle titles are as follows:

- **Rebuilt Title**: Issued when a vehicle has been significantly rebuilt after being declared a total loss.
- **Salvage Title**: Issued when a vehicle has been declared a total loss by an insurance company due to damage that's more expensive to repair than the car's value.

We will **assume** here that the NaN in 'clean_title' are either rebuilt titles and possibly missing information and so we will impute them with *No*

In [ ]:
train_df['clean_title'].fillna('No', inplace=True)

train_df['title_mapped'] = train_df['clean_title'].map({'Yes': 1, 'No': 0})

In [26]:
train_df['title_mapped'].isna()

id
0         False
1         False
2         False
3         False
4         False
          ...  
188528    False
188529    False
188530    False
188531     True
188532    False
Name: title_mapped, Length: 188533, dtype: bool

In [14]:
train_df = train_df.drop(columns='clean_title')
train_df.head(5)

,brand,model,model_year,mileage,ext_col,int_col,accident,price,transmission_cat,horsepower,engine_size,type_of_fuel,title_mapped
id,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Yellow,Gray,None reported,4200,Automatic,172.0,1.6,Gasoline,1.0
1,Lincoln,LS V8,2002,143250,Silver,Beige,At least 1 accident or damage reported,4999,Automatic,252.0,3.9,Gasoline,1.0
2,Chevrolet,Silverado 2500 LT,2002,136731,Blue,Gray,None reported,13900,Automatic,320.0,5.3,Flex Fuel,1.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Black,Black,None reported,45000,Dual,420.0,5.0,Gasoline,1.0
4,Mercedes-Benz,Metris Base,2021,7388,Black,Beige,None reported,97500,Automatic,208.0,2.0,Gasoline,1.0


## Accident

In [15]:
train_df['accident'] = train_df['accident'].map({'At least 1 accident or damage reported': 1,'None reported': 0})
train_df.head(5)

,brand,model,model_year,mileage,ext_col,int_col,accident,price,transmission_cat,horsepower,engine_size,type_of_fuel,title_mapped
id,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Yellow,Gray,0.0,4200,Automatic,172.0,1.6,Gasoline,1.0
1,Lincoln,LS V8,2002,143250,Silver,Beige,1.0,4999,Automatic,252.0,3.9,Gasoline,1.0
2,Chevrolet,Silverado 2500 LT,2002,136731,Blue,Gray,0.0,13900,Automatic,320.0,5.3,Flex Fuel,1.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Black,Black,0.0,45000,Dual,420.0,5.0,Gasoline,1.0
4,Mercedes-Benz,Metris Base,2021,7388,Black,Beige,0.0,97500,Automatic,208.0,2.0,Gasoline,1.0


In [17]:
train_df['transmission_cat'].unique()

array(['Automatic', 'Dual', 'CVT', 'Manual', 'Other'], dtype=object)

In [22]:
train_df[train_df.isna().any(axis=1)]

,brand,model,model_year,mileage,ext_col,int_col,accident,price,transmission_cat,horsepower,engine_size,type_of_fuel,title_mapped
id,,,,,,,,,,,,,
8,Ford,F-150 XLT,2020,38352,Snowflake White Pearl Metallic,Black,0.0,62890,Automatic,NaN,2.7,NaN,1.0
11,Tesla,Model S P100D,2015,81500,Gray,White,0.0,19000,Automatic,NaN,NaN,Electric,1.0
20,Land,Rover Defender S,2023,7978,Santorini Black Metallic,Black,0.0,41998,Automatic,NaN,5.2,NaN,NaN
26,Ford,F-250 XLT,2014,67100,White,Beige,0.0,25900,Automatic,450.0,6.7,NaN,1.0
32,Tesla,Model S P100D,2018,30300,White,White,0.0,64000,Automatic,518.0,NaN,Electric,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188511,Land,Rover Range Rover Velar P380 SE R-Dynamic,2022,15198,Firenze Red,Ebony,0.0,23739,Automatic,NaN,2.0,NaN,NaN
188513,Lexus,GS 350 F Sport,2022,82046,Caviar,Black,0.0,48990,Automatic,NaN,3.5,NaN,NaN
188515,GMC,Sierra 1500 Denali Ultimate,2023,9978,White Frost Tri-Coat,Walnut,1.0,48999,Automatic,NaN,6.2,NaN,1.0
